# Togo Solar Data: EDA, Profiling & Cleaning
Date: November 12, 2025  
Author: Elias Wakgari 
Objective: Profile, clean, and explore Togo's solar dataset (Dapaong Station) for insights.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats  # For Z-scores
import warnings
import os  # For absolute paths
warnings.filterwarnings('ignore')

# Set style for plots
plt.style.use('seaborn-v0_8')  # Or 'default' if not available
sns.set_palette("husl")

# Absolute path
project_root = r'C:\Users\Y\solar-challenge-week1'
file_path = os.path.join(project_root, 'data', 'togo-dapaong_qc.csv')

print(f"🔍 Full file path: {file_path}")
print(f"✅ File exists? {os.path.exists(file_path)}")

# Load raw (no parse_dates—handle in peek)
df = pd.read_csv(file_path)
print(f"Dataset shape: {df.shape}")
print("Columns:", df.columns.tolist())
print("\nFirst 5 rows:\n", df.head())

# Build Timestamp (robust peek-based)
date_cols = ['YEAR', 'MONTH', 'DAY', 'HOUR']
lc_date_cols = [col.lower() for col in date_cols]
possible_single = ['Timestamp', 'DateTime', 'date', 'local_time', 'time']
ts_col = None

if all(col in df.columns for col in date_cols):
    print("✅ Split cols found—building Timestamp.")
    df['Timestamp'] = pd.to_datetime(df[date_cols].assign(MINUTE=0, SECOND=0))
    df.drop(date_cols, axis=1, inplace=True)
elif all(col in df.columns for col in lc_date_cols):
    print("✅ Lowercase split cols—building.")
    df['Timestamp'] = pd.to_datetime(df[lc_date_cols].assign(MINUTE=0, SECOND=0))
    df.drop(lc_date_cols, axis=1, inplace=True)
else:
    ts_col = next((col for col in possible_single if col in df.columns), None)
    if ts_col:
        print(f"✅ Single TS col '{ts_col}'—parsing.")
        df['Timestamp'] = pd.to_datetime(df[ts_col])
        df.drop(ts_col, axis=1, inplace=True)
    else:
        print("❌ No date cols found—check head above and add manual (e.g., 'date_col' = 'your_col').")

# Set index
if 'Timestamp' in df.columns:
    df.set_index('Timestamp', inplace=True)
    print("✅ Timestamp set! Date range:", df.index.min(), "to", df.index.max())
    print(df.head())
    print(df.dtypes)
else:
    print("⚠️ No Timestamp—stop and adjust cols from head.")

🔍 Full file path: C:\Users\Y\solar-challenge-week1\data\togo-dapaong_qc.csv
✅ File exists? True
Dataset shape: (525600, 19)
Columns: ['Timestamp', 'GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'Tamb', 'RH', 'WS', 'WSgust', 'WSstdev', 'WD', 'WDstdev', 'BP', 'Cleaning', 'Precipitation', 'TModA', 'TModB', 'Comments']

First 5 rows:
           Timestamp  GHI  DNI  DHI  ModA  ModB  Tamb    RH   WS  WSgust  \
0  2021-10-25 00:01 -1.3  0.0  0.0   0.0   0.0  24.8  94.5  0.9     1.1   
1  2021-10-25 00:02 -1.3  0.0  0.0   0.0   0.0  24.8  94.4  1.1     1.6   
2  2021-10-25 00:03 -1.3  0.0  0.0   0.0   0.0  24.8  94.4  1.2     1.4   
3  2021-10-25 00:04 -1.2  0.0  0.0   0.0   0.0  24.8  94.3  1.2     1.6   
4  2021-10-25 00:05 -1.2  0.0  0.0   0.0   0.0  24.8  94.0  1.3     1.6   

   WSstdev     WD  WDstdev   BP  Cleaning  Precipitation  TModA  TModB  \
0      0.4  227.6      1.1  977         0            0.0   24.7   24.4   
1      0.4  229.3      0.7  977         0            0.0   24.7   24.4   
2   

In [3]:
# Absolute path
project_root = r'C:\Users\Y\solar-challenge-week1'
file_path = os.path.join(project_root, 'data', 'togo-dapaong_qc.csv')

# Load raw (no parse_dates—handle in peek)
df = pd.read_csv(file_path)
print(f"Dataset shape: {df.shape}")
print("Columns:", df.columns.tolist())
print(df.head())

# Build Timestamp (peek-based)
date_cols = ['YEAR', 'MONTH', 'DAY', 'HOUR']
if all(col in df.columns for col in date_cols):
    df['Timestamp'] = pd.to_datetime(df[date_cols].assign(MINUTE=0, SECOND=0))
    df.drop(date_cols, axis=1, inplace=True)
elif all(col.lower() in df.columns for col in date_cols):
    lc_cols = [col.lower() for col in date_cols]
    df['Timestamp'] = pd.to_datetime(df[lc_cols].assign(MINUTE=0, SECOND=0))
    df.drop(lc_cols, axis=1, inplace=True)
else:
    possible_ts = ['Timestamp', 'date', 'datetime', 'local_time']
    ts_col = next((col for col in possible_ts if col in df.columns), None)
    if ts_col:
        df['Timestamp'] = pd.to_datetime(df[ts_col])
        df.drop(ts_col, axis=1, inplace=True)

df.set_index('Timestamp', inplace=True)
print("Date range:", df.index.min(), "to", df.index.max())
print(df.head())
print(df.dtypes)

Dataset shape: (525600, 19)
Columns: ['Timestamp', 'GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'Tamb', 'RH', 'WS', 'WSgust', 'WSstdev', 'WD', 'WDstdev', 'BP', 'Cleaning', 'Precipitation', 'TModA', 'TModB', 'Comments']
          Timestamp  GHI  DNI  DHI  ModA  ModB  Tamb    RH   WS  WSgust  \
0  2021-10-25 00:01 -1.3  0.0  0.0   0.0   0.0  24.8  94.5  0.9     1.1   
1  2021-10-25 00:02 -1.3  0.0  0.0   0.0   0.0  24.8  94.4  1.1     1.6   
2  2021-10-25 00:03 -1.3  0.0  0.0   0.0   0.0  24.8  94.4  1.2     1.4   
3  2021-10-25 00:04 -1.2  0.0  0.0   0.0   0.0  24.8  94.3  1.2     1.6   
4  2021-10-25 00:05 -1.2  0.0  0.0   0.0   0.0  24.8  94.0  1.3     1.6   

   WSstdev     WD  WDstdev   BP  Cleaning  Precipitation  TModA  TModB  \
0      0.4  227.6      1.1  977         0            0.0   24.7   24.4   
1      0.4  229.3      0.7  977         0            0.0   24.7   24.4   
2      0.3  228.5      2.9  977         0            0.0   24.7   24.4   
3      0.3  229.1      4.6  977         0 

KeyError: "None of ['Timestamp'] are in the columns"